In [162]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.optim as optim
import numpy as np
from diffusers.schedulers.scheduling_ddpm import DDPMScheduler
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
from PIL import Image
import os
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pad_sequence
from torchmetrics import MeanAbsoluteError
from sklearn.preprocessing import StandardScaler
from torchvision.transforms import Resize, ToTensor
from torchvision.transforms.functional import to_pil_image
# from models.resnet import ResNet18VisualEncoder


In [163]:
# Load the trajectory data
def load_trajectory_data(file_path):
    return np.load(file_path, allow_pickle=True)    

In [164]:
# Initialize the DDPM scheduler
noise_scheduler = DDPMScheduler(num_train_timesteps=1000, beta_schedule='squaredcos_cap_v2')

In [165]:
def add_noise_to_trajectory(trajectory, noise_scheduler):
    # Convert the trajectory to a tensor
    trajectory_tensor = torch.tensor(trajectory, dtype=torch.float32)
    Image_min = 2
    Image_max = 99

    noise = torch.randn_like(trajectory_tensor)* 15
    noise = torch.clamp(noise,Image_min, Image_min)
    timesteps = torch.randint(100, 800, (trajectory_tensor.shape[0],)).long()

    # Add noise using the scheduler
    noisy_trajectory = noise_scheduler.add_noise(trajectory_tensor, noise, timesteps)
    noisy_trajectory = torch.clamp(noisy_trajectory,Image_min, Image_max)

    # Return the noisy trajectory as a numpy array
    return noisy_trajectory.numpy()


In [166]:
# Load the trajectory data
trajectory_data = load_trajectory_data("./data/trajectories_1.npy")

In [167]:
# Loop through each trajectory data
noisy_trajectories = []
for data in trajectory_data:
    trajectory = data['trajectory']
    
    # Add noise to the trajectory
    noisy_trajectory = add_noise_to_trajectory(trajectory, noise_scheduler)
    
    # Append the noisy trajectory to the list
    noisy_trajectories.append({
        'map_name': data['map_name'],
        'start': data['start'],
        'goal': data['goal'],
        'noisy_trajectory': noisy_trajectory  # Save the final noisy trajectory
    })

# Save the data to a .npy file
output_file_path = './data/noisy_trajectories_1.npy'
np.save(output_file_path, noisy_trajectories)

output_file_path

'./data/noisy_trajectories_1.npy'